In [68]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Population by state

In [69]:

URL_DATA = './staging/bronze/state_population_bronze_data.parquet'

state_population = pd.read_parquet(URL_DATA)

if 'date' in state_population.columns:
    
    state_population['date'] = pd.to_datetime(state_population['date'])
    
    state_population['year'] = state_population['date'].dt.year

    state_population.drop(columns='date', inplace=True)


state_population = state_population[
    (state_population['sex'] == 'both') &
    (state_population['age'] == 'overall') & 
    (state_population['ethnicity'] == 'overall')
]

state_population['population'] = (state_population['population'] * 1000).astype(int)

state_population = state_population[["year", "state", "population"]]

state_population = state_population.rename(columns={
    'population': 'population_state'
})

state_population = state_population[~state_population['state'].isin(['W.P. Labuan', 'W.P. Putrajaya'])]

state_population['state'] = state_population['state'].replace({
    'W.P. Kuala Lumpur': 'Kuala Lumpur',
})

state_population 

,year,state,population_state
0,1970,Johor,1325600
48,1971,Johor,1355400
96,1972,Johor,1385300
144,1973,Johor,1414500
192,1974,Johor,1444400
...,...,...,...
241335,2021,Kuala Lumpur,1964000
241734,2022,Kuala Lumpur,1961200
242133,2023,Kuala Lumpur,2005700
242532,2024,Kuala Lumpur,2067500


In [70]:
# save data
state_population.to_parquet("./staging/silver/state_population_silver_data.parquet", index=False)

## Population by national

In [71]:
URL_DATA = './staging/bronze/national_population_bronze_data.parquet'

national_population = pd.read_parquet(URL_DATA)

if 'date' in national_population.columns:
    
    national_population['date'] = pd.to_datetime(national_population['date'])
    
    national_population['year'] = national_population['date'].dt.year

    national_population.drop(columns='date', inplace=True)

national_population = national_population[
    (national_population['sex'] == 'both') &
    (national_population['age'] == 'overall') & 
    (national_population['ethnicity'] == 'overall')
]

national_population['population'] = (national_population['population'] * 1000).astype(int)

national_population = national_population[["year", "population"]]

national_population = national_population.rename(columns={
    'population': 'population_national',
})

national_population

,year,population_national
0,1970,10881800
48,1971,11159700
96,1972,11441300
144,1973,11719800
192,1974,12001300
240,1975,12300300
288,1976,12588100
336,1977,12901100
384,1978,13200200
432,1979,13518300


In [72]:
# save data
national_population.to_parquet("./staging/silver/national_population_silver_data.parquet", index=False)

## Inflation rate national

In [73]:

URL_DATA = './staging/bronze/inflation_national_bronze_data.parquet'

inflation_national = pd.read_parquet(URL_DATA)

if 'date' in inflation_national.columns:
    
    inflation_national['date'] = pd.to_datetime(inflation_national['date'])
    
    inflation_national['year'] = inflation_national['date'].dt.year

    inflation_national.drop(columns='date', inplace=True)
    
inflation_national = inflation_national[
    (inflation_national['division'] == 'overall')
]    

inflation_national = inflation_national[["year", "inflation"]]

inflation_national = inflation_national.rename(columns={
    'inflation': 'Inflation(%)'
})

inflation_national['Inflation(%)'] = inflation_national['Inflation(%)'].round(1)



inflation_national


,year,Inflation(%)
0,1961,-0.2
1,1962,0.1
2,1963,3.1
3,1964,-0.4
4,1965,-0.1
...,...,...
59,2020,-1.1
60,2021,2.5
61,2022,3.4
62,2023,2.5


In [74]:
# save data
inflation_national.to_parquet("./staging/silver/inflation_national_silver_data.parquet", index=False)

## Annual Real GDP by State & Economic Sector

In [75]:
URL_DATA = './staging/bronze/gdp_state_bronze_data.parquet'

gdp_state = pd.read_parquet(URL_DATA)

if 'date' in gdp_state.columns:
    
    gdp_state['date'] = pd.to_datetime(gdp_state['date'])
    
    gdp_state['year'] = gdp_state['date'].dt.year

    gdp_state.drop(columns='date', inplace=True)
    

gdp_state = gdp_state[
    (gdp_state['series'] == 'abs') &
    (gdp_state['sector'] == 'p0') &  
    (gdp_state['state'] != 'Supra') 
]

gdp_state = gdp_state.rename(columns={
    'value': 'gdp_per_state(RM)'
})

gdp_state['gdp_per_state(RM)'] = (gdp_state['gdp_per_state(RM)'] * 1000000).astype(int)

gdp_state = gdp_state[["year","state", "gdp_per_state(RM)"]]

gdp_state = gdp_state[~gdp_state['state'].isin(['W.P. Labuan', 'W.P. Putrajaya'])]

gdp_state['state'] = gdp_state['state'].replace({
    'W.P. Kuala Lumpur': 'Kuala Lumpur',
})

gdp_state 

,year,state,gdp_per_state(RM)
0,2015,Johor,110002221000
7,2015,Kedah,39549581000
14,2015,Kelantan,21407736000
21,2015,Melaka,36077257000
28,2015,Negeri Sembilan,40185677000
...,...,...,...
903,2023,Sabah,83196915000
910,2023,Sarawak,142351830000
917,2023,Selangor,406071453000
924,2023,Terengganu,38201414000


In [76]:
test = gdp_state[gdp_state["year"] == 2022]
test

,year,state,gdp_per_state(RM)
735,2022,Johor,142369692000
742,2022,Kedah,50982123000
749,2022,Kelantan,26934566000
756,2022,Melaka,45579035000
763,2022,Negeri Sembilan,50939264000
770,2022,Pahang,61746204000
777,2022,Perak,80426886000
784,2022,Perlis,6209957000
791,2022,Pulau Pinang,112281052000
798,2022,Sabah,82106428000


In [77]:
# save data
gdp_state.to_parquet("./staging/silver/gdp_state_silver_data.parquet", index=False)

## Annual Real GDP by national & Economic Sector

In [78]:


URL_DATA = './staging/bronze/gdp_national_bronze_data.parquet'

gdp_national = pd.read_parquet(URL_DATA)

if 'date' in gdp_national.columns:
    
    gdp_national['date'] = pd.to_datetime(gdp_national['date'])
    
    gdp_national['year'] = gdp_national['date'].dt.year

    gdp_national.drop(columns='date', inplace=True)

gdp_national = gdp_national[
    (gdp_national['series'] == 'abs')
]

gdp_national = gdp_national.rename(columns={
    'gdp': 'gdp_national(RM)',
    'gdp_capita': 'gdp_national_per_capita(RM)',
})

gdp_national['gdp_national(RM)'] = (gdp_national['gdp_national(RM)'] * 1_000_000).astype(int)
gdp_national['gdp_national_per_capita(RM)'] = (gdp_national['gdp_national_per_capita(RM)'] * 1_000_000).astype(int)

gdp_national = gdp_national[["year", "gdp_national(RM)", "gdp_national_per_capita(RM)"]]

gdp_national

,year,gdp_national(RM),gdp_national_per_capita(RM)
0,1970,73709682000,6773390000
1,1971,81106199000,7265660000
2,1972,88720810000,7751699000
3,1973,99102104000,8455832000
4,1974,107346073000,8941871000
5,1975,108205988000,8798552000
6,1976,120718377000,9589923000
7,1977,130077743000,10082193000
8,1978,138732976000,10509519000
9,1979,151703260000,11223580000


In [79]:
# save data
gdp_national.to_parquet("./staging/silver/gdp_national_silver_data.parquet", index=False)

## Labour force by state

In [80]:
URL_DATA = './staging/bronze/Labour_force_state_bronze_data.parquet'

Labour_force_state = pd.read_parquet(URL_DATA)

if 'date' in Labour_force_state.columns:
    
    Labour_force_state['date'] = pd.to_datetime(Labour_force_state['date'])
    
    Labour_force_state['year'] = Labour_force_state['date'].dt.year

    Labour_force_state.drop(columns='date', inplace=True)
    
Labour_force_state = Labour_force_state[
    (Labour_force_state['sex'] == 'both') 
]

Labour_force_state = Labour_force_state[["year", "state", "lf", "lf_employed", "lf_unemployed", "lf_outside", "u_rate"]].rename(columns={
    "lf": "labour_force_state",
    "lf_employed": "employed_persons_state",
    "lf_unemployed": "unemployed_persons_state",
    "lf_outside": "outside_labour_force_state",
    "u_rate": "unemployed_rate_state(%)"
})


# # interpolation
# # ===================================================


# Step 1: Define full year range and get all states
year_range = list(range(1982, 2024))
all_states = Labour_force_state['state'].unique()

# Step 2: Create full multi-index of (state, year)
full_index = pd.MultiIndex.from_product(
    [all_states, year_range], names=['state', 'year']
)

# Step 3: Set multi-index and reindex to fill missing years
Labour_force_state = Labour_force_state.set_index(['state', 'year']).reindex(full_index)

# Step 4: Interpolate missing values per state
Labour_force_state = Labour_force_state.groupby(level=0, group_keys=False).apply(
    lambda group: group.interpolate(method='linear', limit_direction='both')
)

# Step 5: Reset index cleanly
Labour_force_state = Labour_force_state.reset_index()



# multiply by 1000
cols_to_expand = ['labour_force_state', 'employed_persons_state', 'unemployed_persons_state', 'outside_labour_force_state']

Labour_force_state[cols_to_expand] = Labour_force_state[cols_to_expand].apply(lambda x: (x * 1000).astype(int))

# round off unemployed_rate
Labour_force_state['unemployed_rate_state(%)'] = Labour_force_state['unemployed_rate_state(%)'].round(0)

Labour_force_state = Labour_force_state[~Labour_force_state['state'].isin(['W.P. Labuan', 'W.P. Putrajaya'])]

Labour_force_state['state'] = Labour_force_state['state'].replace({
    'W.P. Kuala Lumpur': 'Kuala Lumpur',
})

Labour_force_state

,state,year,labour_force_state,employed_persons_state,unemployed_persons_state,outside_labour_force_state,unemployed_rate_state(%)
0,Johor,1982,653100,630200,22900,345100,4.0
1,Johor,1983,650900,623700,27200,344100,4.0
2,Johor,1984,688200,657300,30900,372100,4.0
3,Johor,1985,710400,672700,37600,354700,5.0
4,Johor,1986,751200,700400,50800,367800,7.0
...,...,...,...,...,...,...,...
583,Kuala Lumpur,2019,863400,841000,22400,369600,3.0
584,Kuala Lumpur,2020,1049500,1006400,43100,408300,4.0
585,Kuala Lumpur,2021,1040700,992500,48300,414000,5.0
586,Kuala Lumpur,2022,1060300,1019700,40500,397300,4.0


In [81]:
tes = Labour_force_state[Labour_force_state["state"] == "Johor"]
tes

,state,year,labour_force_state,employed_persons_state,unemployed_persons_state,outside_labour_force_state,unemployed_rate_state(%)
0,Johor,1982,653100,630200,22900,345100,4.0
1,Johor,1983,650900,623700,27200,344100,4.0
2,Johor,1984,688200,657300,30900,372100,4.0
3,Johor,1985,710400,672700,37600,354700,5.0
4,Johor,1986,751200,700400,50800,367800,7.0
5,Johor,1987,781800,732700,49100,380100,6.0
6,Johor,1988,837000,792000,45100,396500,5.0
7,Johor,1989,861800,822300,39500,439400,5.0
8,Johor,1990,949100,914100,35100,449600,4.0
9,Johor,1991,948100,920050,28100,451800,3.0


In [82]:
# save data
Labour_force_state.to_parquet("./staging/silver/Labour_force_state_silver_data.parquet", index=False)

# Labour force by national

In [83]:

URL_DATA = './staging/bronze/Labour_force_national_bronze_data.parquet'

Labour_force_national = pd.read_parquet(URL_DATA)

if 'date' in Labour_force_national.columns:
    
    Labour_force_national['date'] = pd.to_datetime(Labour_force_national['date'])
    
    Labour_force_national['year'] = Labour_force_national['date'].dt.year

    Labour_force_national.drop(columns='date', inplace=True)

Labour_force_national = Labour_force_national[["year","lf", "lf_employed", "lf_unemployed", "lf_outside", "u_rate", "p_rate"]].rename(columns={
    "lf": "labour_force_national",
    "lf_employed": "employed_persons_national",
    "lf_unemployed": "unemployed_persons_national",
    "lf_outside": "outside_labour_force_national",
    "u_rate": "unemployed_rate_national(%)",
    "p_rate": "participation_rate_national(%)"
})

# interpolation
# ===================================================

# Ensure year is the index BEFORE reindexing
Labour_force_national = Labour_force_national.set_index('year')

# Create full year range
full_years = pd.Index(range(1982, 2023), name='year')

# Reindex to full year range
Labour_force_national = Labour_force_national.reindex(full_years)

# Define columns to interpolate
col = [
    "labour_force_national",
    "employed_persons_national",
    "unemployed_persons_national",
    "outside_labour_force_national",
    "unemployed_rate_national(%)",
    "participation_rate_national(%)"
]

# Interpolate only existing columns
Labour_force_national[col] = Labour_force_national[col].interpolate(method='linear')


# multiply by 1000
cols_to_expand = ['labour_force_national', 'employed_persons_national', 'unemployed_persons_national', 'outside_labour_force_national']

Labour_force_national[cols_to_expand] = Labour_force_national[cols_to_expand].apply(lambda x: (x * 1000).astype(int))

# round off unemployed_rate
Labour_force_national['unemployed_rate_national(%)'] = Labour_force_national['unemployed_rate_national(%)'].round(0)

# Reset index to bring 'year' back as a column
Labour_force_national = Labour_force_national.reset_index()

Labour_force_national

,year,labour_force_national,employed_persons_national,unemployed_persons_national,outside_labour_force_national,unemployed_rate_national(%),participation_rate_national(%)
0,1982,5431400,5249000,182400,2944600,3.0,64.8
1,1983,5671800,5457000,214900,2969400,4.0,65.6
2,1984,5862500,5566700,295800,3119600,5.0,65.3
3,1985,5990100,5653400,336800,3124900,6.0,65.7
4,1986,6222100,5760100,461900,3188300,7.0,66.1
5,1987,6456800,5983900,472900,3246100,7.0,66.5
6,1988,6637000,6157200,479800,3301500,7.0,66.8
7,1989,6779400,6390900,388500,3463500,6.0,66.2
8,1990,7000200,6685000,315200,3519700,4.0,66.5
9,1991,7159600,6866400,293200,3651649,4.0,66.2


In [84]:
# save data
Labour_force_national.to_parquet("./staging/silver/Labour_force_national_silver_data.parquet", index=False)

# Household income by state

In [85]:
URL_DATA = './staging/bronze/house_hold_income_state_bronze_data.parquet'

house_hold_income_state = pd.read_parquet(URL_DATA)

if 'date' in house_hold_income_state.columns:
    
    house_hold_income_state['date'] = pd.to_datetime(house_hold_income_state['date'])
    
    house_hold_income_state['year'] = house_hold_income_state['date'].dt.year

    house_hold_income_state.drop(columns='date', inplace=True)
    
    house_hold_income_state.set_index('year', inplace=True)
    
house_hold_income_state = house_hold_income_state.rename(columns={
    'income_mean': 'income_mean_state(RM)',
    'income_median': 'income_median_state(RM)'
})

# fill in the NA with 269.0
house_hold_income_state.fillna(269.0, inplace=True)

# # interpolation
# # ===================================================

# Assuming 'house_hold_income_state' is already loaded
house_hold_income_state = house_hold_income_state.reset_index()  # In case 'year' is index

# Step 1: Define full year range and get all states
year_range = list(range(1970, 2023))
all_states = house_hold_income_state['state'].unique()

# Step 2: Create full multi-index of (state, year)
full_index = pd.MultiIndex.from_product(
    [all_states, year_range], names=['state', 'year']
)

# Step 3: Set multi-index and reindex to fill missing years
house_hold_income_state = house_hold_income_state.set_index(['state', 'year']).reindex(full_index)

# Step 4: Interpolate missing values per state
house_hold_income_state = house_hold_income_state.groupby(level=0, group_keys=False).apply(
    lambda group: group.interpolate(method='linear', limit_direction='both')
)

# Step 5: Reset index cleanly
house_hold_income_state = house_hold_income_state.reset_index()

house_hold_income_state[['income_mean_state(RM)', 'income_median_state(RM)']] = house_hold_income_state[['income_mean_state(RM)', 'income_median_state(RM)']].round(1)

house_hold_income_state = house_hold_income_state[["year", "state", "income_mean_state(RM)","income_median_state(RM)"]]

house_hold_income_state = house_hold_income_state[~house_hold_income_state['state'].isin(['W.P. Labuan', 'W.P. Putrajaya'])]

house_hold_income_state['state'] = house_hold_income_state['state'].replace({
    'W.P. Kuala Lumpur': 'Kuala Lumpur',
})

house_hold_income_state


,year,state,income_mean_state(RM),income_median_state(RM)
0,1970,Johor,237.0,269.0
1,1971,Johor,273.2,269.0
2,1972,Johor,309.5,269.0
3,1973,Johor,345.8,269.0
4,1974,Johor,382.0,269.0
...,...,...,...,...
737,2018,Kuala Lumpur,12735.3,10057.0
738,2019,Kuala Lumpur,13257.0,10549.0
739,2020,Kuala Lumpur,11728.0,9093.0
740,2021,Kuala Lumpur,12526.5,9663.5


In [86]:
house_hold_income_state.isna().any().any()

np.False_

In [87]:
# testing data from each state
test = house_hold_income_state[house_hold_income_state['state'] == 'Sabah']
test

,year,state,income_mean_state(RM),income_median_state(RM)
477,1970,Sabah,513.0,309.0
478,1971,Sabah,513.0,309.0
479,1972,Sabah,513.0,309.0
480,1973,Sabah,513.0,309.0
481,1974,Sabah,513.0,309.0
482,1975,Sabah,513.0,309.0
483,1976,Sabah,513.0,309.0
484,1977,Sabah,597.7,370.3
485,1978,Sabah,682.3,431.7
486,1979,Sabah,767.0,493.0


In [88]:
# save data
house_hold_income_state.to_parquet("./staging/silver/house_hold_income_state_silver_data.parquet", index=False)

# Household income by national

In [89]:
# get the latest national mean income

URL_DATA = './staging/bronze/house_hold_income_national_bronze_data.parquet'
house_hold_income_national = pd.read_parquet(URL_DATA)

if 'date' in house_hold_income_national.columns: house_hold_income_national['date'] = pd.to_datetime(house_hold_income_national['date'])

if 'date' in house_hold_income_national.columns:
    
    house_hold_income_national['date'] = pd.to_datetime(house_hold_income_national['date'])
    
    house_hold_income_national['year'] = house_hold_income_national['date'].dt.year

    house_hold_income_national.drop(columns='date', inplace=True)
    
    house_hold_income_national.set_index('year', inplace=True)
    
house_hold_income_national = house_hold_income_national.rename(columns={
    'income_mean': 'income_mean_national(RM)',
    'income_median': 'income_median_national(RM)'
})

# interpolation
# ===================================================
# Step 1: Create full year range
full_years = pd.Index(range(1970, 2023), name='year')

# Step 2: Reindex and interpolate (no need for groupby since national data isn't by state)
house_hold_income_national = house_hold_income_national.reindex(full_years)
house_hold_income_national['income_mean_national(RM)'] = house_hold_income_national['income_mean_national(RM)'].interpolate(method='linear')
house_hold_income_national['income_median_national(RM)'] = house_hold_income_national['income_median_national(RM)'].interpolate(method='linear')

house_hold_income_national[['income_mean_national(RM)', 'income_median_national(RM)']] = house_hold_income_national[['income_mean_national(RM)', 'income_median_national(RM)']].round(1)

house_hold_income_national.reset_index('year', inplace=True)

house_hold_income_national


,year,income_mean_national(RM),income_median_national(RM)
0,1970,264.0,166.0
1,1971,288.5,181.2
2,1972,313.0,196.5
3,1973,337.5,211.8
4,1974,362.0,227.0
5,1975,433.5,267.5
6,1976,505.0,308.0
7,1977,562.7,348.3
8,1978,620.3,388.7
9,1979,678.0,429.0


In [90]:
# save data
house_hold_income_national.to_parquet("./staging/silver/house_hold_income_national_silver_data.parquet", index=False)

## Household Expenditure state

In [91]:
URL_DATA = './staging/bronze/house_hold_expenditure_bronze_data.parquet'
house_hold_expenditure = pd.read_parquet(URL_DATA)

# Interpolation 
# ===========================================

# Step 1: Define full year range and get all states
year_range = list(range(2016, 2023))
all_states = house_hold_expenditure['state'].unique()

# Step 2: Create full multi-index of (state, year)
full_index = pd.MultiIndex.from_product(
    [all_states, year_range], names=['state', 'year']
)

# Step 3: Set multi-index and reindex to fill missing years
house_hold_expenditure = house_hold_expenditure.set_index(['state', 'year']).reindex(full_index)

# Step 4: Interpolate missing values per state
house_hold_expenditure = house_hold_expenditure.groupby(level=0, group_keys=False).apply(
    lambda group: group.interpolate(method='linear', limit_direction='both')
)

# Step 5: Reset index cleanly
house_hold_expenditure = house_hold_expenditure.reset_index()

house_hold_expenditure[['expenditure_mean_state(RM)', 'expenditure_median_state(RM)']] = house_hold_expenditure[['expenditure_mean_state(RM)', 'expenditure_median_state(RM)']].round(1)

house_hold_expenditure = house_hold_expenditure[
    ["year", "state", "expenditure_mean_state(RM)", "expenditure_median_state(RM)"]
]

house_hold_expenditure = house_hold_expenditure[~house_hold_expenditure['state'].isin(['W.P. Labuan', 'W.P. Putrajaya'])]

house_hold_expenditure['state'] = house_hold_expenditure['state'].replace({
    'W.P. Kuala Lumpur': 'Kuala Lumpur',
})

house_hold_expenditure


,year,state,expenditure_mean_state(RM),expenditure_median_state(RM)
0,2016,Johor,4167.0,3635.0
1,2017,Johor,4386.7,3769.3
2,2018,Johor,4606.3,3903.7
3,2019,Johor,4826.0,4038.0
4,2020,Johor,4998.0,4237.3
...,...,...,...,...
107,2018,Sabah,2733.0,2223.0
108,2019,Sabah,2799.0,2279.0
109,2020,Sabah,2980.0,2442.7
110,2021,Sabah,3161.0,2606.3


In [92]:
# testing data from each state
test = house_hold_expenditure[house_hold_expenditure['year'] == 2022]
test

,year,state,expenditure_mean_state(RM),expenditure_median_state(RM)
6,2022,Johor,5342.0,4636.0
13,2022,Kedah,3765.0,3298.0
20,2022,Kelantan,3505.0,3048.0
27,2022,Melaka,5707.0,4861.0
34,2022,Negeri Sembilan,4678.0,3869.0
41,2022,Pahang,4107.0,3513.0
48,2022,Perak,3903.0,3341.0
55,2022,Pulau Pinang,5322.0,4481.0
62,2022,Perlis,3834.0,3355.0
69,2022,Selangor,6770.0,5747.0


In [93]:
# save data
house_hold_expenditure.to_parquet("./staging/silver/house_hold_expenditure_silver_data.parquet", index=False)

# Relative Poverty compare to state household median income

In [94]:
# Proportion of households with monthly income below half the state median income

URL_DATA = './staging/bronze/house_hold_poverty_state_bronze_data.parquet'

house_hold_poverty_state = pd.read_parquet(URL_DATA)

if 'date' in house_hold_poverty_state.columns:
    
    house_hold_poverty_state['date'] = pd.to_datetime(house_hold_poverty_state['date'])
    
    house_hold_poverty_state['year'] = house_hold_poverty_state['date'].dt.year

    house_hold_poverty_state.drop(columns='date', inplace=True)
    
    house_hold_poverty_state.set_index('year', inplace=True)
    
house_hold_poverty_state = house_hold_poverty_state.reset_index()
house_hold_poverty_state = house_hold_poverty_state[['year', 'state', 'poverty_relative']]
house_hold_poverty_state = house_hold_poverty_state.rename(columns={'poverty_relative': 'poverty_relative_state_median_income(%)'})

house_hold_poverty_state = house_hold_poverty_state.dropna()


# interpolation
# ========================================


# Generate full index
year_range = list(range(2016, 2023))
all_states = house_hold_poverty_state['state'].unique()

full_index = pd.MultiIndex.from_product(
    [all_states, year_range], names=['state', 'year']
)

# Reindex and interpolate
house_hold_poverty_state = house_hold_poverty_state.set_index(['state', 'year']).reindex(full_index)

house_hold_poverty_state = house_hold_poverty_state.groupby(level=0, group_keys=False).apply(
    lambda group: group.interpolate(method='linear', limit_direction='both')
)

# Final clean index
house_hold_poverty_state = house_hold_poverty_state.reset_index()


# Round to 2 decimal places
house_hold_poverty_state['poverty_relative_state_median_income(%)'] = house_hold_poverty_state['poverty_relative_state_median_income(%)'].round(0)

house_hold_poverty_state = house_hold_poverty_state[~house_hold_poverty_state['state'].isin(['W.P. Labuan', 'W.P. Putrajaya'])]

house_hold_poverty_state['state'] = house_hold_poverty_state['state'].replace({
    'W.P. Kuala Lumpur': 'Kuala Lumpur',
})

house_hold_poverty_state


,state,year,poverty_relative_state_median_income(%)
0,Johor,2016,14.0
1,Johor,2017,14.0
2,Johor,2018,15.0
3,Johor,2019,15.0
4,Johor,2020,14.0
...,...,...,...
93,Kuala Lumpur,2018,12.0
94,Kuala Lumpur,2019,11.0
95,Kuala Lumpur,2020,9.0
96,Kuala Lumpur,2021,11.0


In [95]:
# testing data from each state
test = house_hold_poverty_state[house_hold_poverty_state['state'] == 'Selangor']
test

,state,year,poverty_relative_state_median_income(%)
77,Selangor,2016,11.0
78,Selangor,2017,12.0
79,Selangor,2018,14.0
80,Selangor,2019,15.0
81,Selangor,2020,14.0
82,Selangor,2021,14.0
83,Selangor,2022,14.0


In [96]:
house_hold_poverty_state

,state,year,poverty_relative_state_median_income(%)
0,Johor,2016,14.0
1,Johor,2017,14.0
2,Johor,2018,15.0
3,Johor,2019,15.0
4,Johor,2020,14.0
...,...,...,...
93,Kuala Lumpur,2018,12.0
94,Kuala Lumpur,2019,11.0
95,Kuala Lumpur,2020,9.0
96,Kuala Lumpur,2021,11.0


In [97]:
# save data
house_hold_poverty_state.to_parquet("./staging/silver/house_hold_poverty_state_silver_data.parquet", index=False)

## Relative Poverty compare to national household median income

In [98]:
URL_DATA = './staging/bronze/house_hold_poverty_national_bronze_data.parquet'


house_hold_poverty_national = pd.read_parquet(URL_DATA)

if 'date' in house_hold_poverty_national.columns:
    
    house_hold_poverty_national['date'] = pd.to_datetime(house_hold_poverty_national['date'])
    
    house_hold_poverty_national['year'] = house_hold_poverty_national['date'].dt.year

    house_hold_poverty_national.drop(columns='date', inplace=True)
    
    house_hold_poverty_national.set_index('year', inplace=True)
    
house_hold_poverty_national.reset_index('year', inplace=True)

house_hold_poverty_national = house_hold_poverty_national[["year", "poverty_relative"]]


house_hold_poverty_national = house_hold_poverty_national.rename(columns={
    'poverty_relative': 'poverty_relative_national_median_income(%)'
})

# interpolation
# ========================================

# Step 1: Set 'year' as index
house_hold_poverty_national = house_hold_poverty_national.set_index('year')

# Step 2: Create full year index from 2016 to 2022
full_years = pd.Index(range(1995, 2023), name='year')

# Step 3: Reindex to include missing years
house_hold_poverty_national = house_hold_poverty_national.reindex(full_years)

# Step 4: Interpolate poverty values
house_hold_poverty_national['poverty_relative_national_median_income(%)'] = house_hold_poverty_national['poverty_relative_national_median_income(%)'].interpolate(method='linear')

# Optional: Reset index back to column if needed
house_hold_poverty_national = house_hold_poverty_national.reset_index()

house_hold_poverty_national['poverty_relative_national_median_income(%)'] = house_hold_poverty_national['poverty_relative_national_median_income(%)'].round(0)


house_hold_poverty_national

,year,poverty_relative_national_median_income(%)
0,1995,20.0
1,1996,20.0
2,1997,20.0
3,1998,19.0
4,1999,19.0
5,2000,19.0
6,2001,20.0
7,2002,20.0
8,2003,20.0
9,2004,20.0


In [99]:
# save data
house_hold_poverty_national.to_parquet("./staging/silver/house_hold_poverty_national_silver_data.parquet", index=False)